# Starbucks Capstone Project

# 0.0 Project Introduction


This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

## 1.0 CRISP-DM

###      1.1 Business Understanding

Starbucks has an mobile app where registered users can use it to order coffe for pickup while mobile, pay in store using a pre-paid function, and collect rewards points.   This app also offers promotions for bonus points to these users.   Starbucks would like to know which of thier promotions are most efficent in inspiring purchases.

The primary task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. 

### 1.2 Data Understanding

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


   ### 1.3 Prepare Data


   #### 1.3.1 Cleaning

##### *Cleaning Portfolio*
1. rename "offer id" to "offer_id" to match transcript
2. one-hot encode "channels" and "offer_type"
    - channels = []
    - offer types = []
3.  ~~set index to the offer id for easy reference~~ **use original index, reset transaction user for easy reading**

##### *Cleaning Profile*
1. became member on -> to_datetime
2. Drop nas - 13% of data is dropped becuase of missing data
3.  one-hot encoded gender
4.  set index as id (user_id) for easier reference

##### *Cleaning Transcript*
1. convert value dict into columns based on key.  Differnt value per offer

| event | value |
|-------|-------|
| offer rcvd | 'offer id'       |
| offer viewed  | 'offer id'       |
| trans |       | 'amount'
| compl | 'offer_id' , 'reward'     |

2. merge offer_id column 
3. drop columns not needed
4. reorder columns
5. only include persons left on profile
6. split into 4 additional dfs by event type



#### 1.3.2 Observations

##### *Portfolio Observation*
* 10 - Offers

##### *Profile Observations*
* 14825/17000 profiles (87% with no na)

##### *Transcript Observations*
* time is by hour.  Time window is about 30 days (714 hrs or 29.75 days)
* 4 types of transactions - [offer,view,transaction,complete]
    * transactions are not specifically tied to offers.
* offers are not opt-in - they need to be *viewed* to be completed
* transactions that coorespond to reward have the same timestamp as the reward
* offers can be made more than once to the same person
* offers do not always result in a reward 

#### 1.3.3 Thoughts


##### *Profile Thoughts*
- Need to perform some demographic baseline

##### *Transaction Thoughts*
* I need to take two tacts on this.
    - [ ] Clean as one block
    - [X] Split data into 4 distict df based on transaction types for matching.
        - [X] find a way to match events from each list with each other


#### 1.3.3 To-Do

##### *Profile To-Do*
- [ ] Revisit *drop_nas* (13% of 
    * [ ]  Compare spend, significant difference?
- [X] Histograms
    - [X] income - overall
    - [X] " by gender
    - [X] " by age
    - [X] " length of membership
    - [X] Make prettier Histograms
    - [X] KDE income by age
    - [X] KDA income by length of membership
    - [ ] Color KDE by groupings

##### *Transcript To-Do*
- [ ] While cleaning, replace user_id with index id from user matrix for easier viewing
- [X] Create user_offer matrix from t_offer. Do individuals get same offer more than once (YES)
- [ ] Explore breakdown of completed by type
    - [ ] Search for coorelations between offer type, extent of response, and known demographic.
    

### 1.3.4 Demographic Analysis

- There are 4 avaible demographic metrics
    - Age  ( continuous)
    - Gender ( Discrete)
    - Income (continuous) 
    - Length of time as a member (continuous)
    
- Of the three continuous variables, it appears none are normally distributeed but rather there are muliple peaks suggesting groupings.
- None of the aforementioned groupings seemed to be related to either of the three other varibable

### 1.4 Data Modeling

#### 1.4.1 *To-Do*

- [ ] 

   ### 1.5 Evaluate Results

   ### 1.6 Deploy